In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import AntData, ForagerData, ExperimentData, InteractionData
from os import sep as sep

In [2]:
exp11 = ExperimentData(11, bdata_path='blob analysis normalized by white paper')

In [3]:
interactions_df = exp11.interactions_df
interactions_df.head()

,vidnum,id,actual_ant1,actual_ant2,actual_start,actual_end,group,general_start_frame,general_end_frame,general_group_id
1,1.0,29.0,392,76.0,116.0,159.0,NaN,72.0,115.0,NaN
2,1.0,35.0,82,169.0,195.0,197.0,NaN,151.0,153.0,NaN
3,1.0,44.0,169,529.0,240.0,243.0,NaN,196.0,199.0,NaN
4,1.0,48.0,76,148.0,276.0,281.0,NaN,232.0,237.0,NaN
5,1.0,49.0,180,148.0,288.0,439.0,NaN,244.0,395.0,NaN


In [4]:
clean_crops = exp11.clean_crops
clean_crops.head()

1        1069        1125        1152         139         ...  535  \
   red yellow  red yellow  red yellow  red yellow  red yellow  ...  red   
0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  ...  0.0   
1  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  ...  0.0   
2  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  ...  0.0   
3  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  ...  0.0   
4  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  ...  0.0   

           35         291         171         142         
  yellow  red yellow  red yellow  red yellow  red yellow  
0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  
1    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  
2    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  
3    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  
4    0.0  0.0    0.0  0.0    0.0  0.0    0.0  0.0    0.0  

[5 rows x 132 columns]

In [5]:
bdata = exp11.bdata
bdata.head()

,Unnamed: 0,frame,time,acquisition,a1-angle,a1-crop_area,a1-crop_intensity,a1-error,a1-original_x,a1-original_y,...,a1299-x,a1299-y,a1247-angle,a1247-crop_area,a1247-crop_intensity,a1247-error,a1247-original_x,a1247-original_y,a1247-x,a1247-y
0,0,44,61.608,BLGF,-53.014075,0.0,0.000000,0.0,4866.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,45,62.998,GLRF,-53.145187,12.0,688.251648,0.0,4866.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,46,64.400,BLGF,-53.014075,0.0,0.000000,1.0,4866.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,47,65.808,GLRF,-53.145187,11.0,624.857544,0.0,4867.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,48,67.208,BLGF,-53.145187,0.0,0.000000,0.0,4867.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
trop = InteractionData(interactions_df.iloc[0], bdata, clean_crops, max(clean_crops.index))

In [ ]:
def get_ant_measurements_before(ant_id, trop, bdata, clean_crops):
    ant = AntData(ant_id, bdata)
    ant_before_all = clean_crops.loc[0:trop.start_frame-1,ant_id]
    
    diff_before_yellow = ant_before_all['yellow'].diff()
    diff_before_red = ant_before_all['red'].diff()
    diff_before_yellow.iloc[0]=0
    diff_before_red.iloc[0]=0
    
    red_changes_before = diff_before_red!=0
    if red_changes_before.any():
        last_change_before_red = max(ant_before_all['red'][red_changes_before].index)
    else:
        last_change_before_red = 0
    
    yellow_changes_before = diff_before_yellow!=0
    if yellow_changes_before.any():
        last_change_before_yellow = max(ant_before_all['yellow'][yellow_changes_before].index)
    else:
        last_change_before_yellow = 0
    
    raw_before_dict={}
    raw_before_dict['yellow']=ant.crop_dict_raw['yellow'].loc[last_change_before_yellow:trop.start_frame]
    raw_before_dict['red']=ant.crop_dict_raw['red'].loc[last_change_before_red:trop.start_frame]
    
    return raw_before_dict

In [7]:
def get_ant_measurements_after(ant_id, trop, bdata, clean_crops):
    ant = AntData(ant_id, bdata)
    ant_after_all = clean_crops.loc[trop.end_frame+1:,ant_id]
    
    diff_after_yellow = ant_after_all['yellow'].diff()
    diff_after_red = ant_after_all['red'].diff()
    diff_after_yellow.iloc[0]=0
    diff_after_red.iloc[0]=0
    
    red_changes_after = diff_after_red!=0
    if red_changes_after.any():
        first_change_after_red = min(ant_after_all['red'][red_changes_after].index)
    else:
        first_change_after_red = -1
    
    yellow_changes_after = diff_after_yellow!=0
    if yellow_changes_after.any():
        first_change_after_yellow = min(ant_after_all['yellow'][yellow_changes_after].index)
    else:
        first_change_after_yellow = -1
    
    raw_after_dict={}
    raw_after_dict['yellow']=ant.crop_dict_raw['yellow'].loc[trop.end_frame:first_change_after_yellow]
    raw_after_dict['red']=ant.crop_dict_raw['red'].loc[trop.end_frame:first_change_after_red]
    
    return raw_after_dict

In [19]:
def get_ant_measurements_before_or_after(ant_id, trop, bdata, clean_crops, direction):
    ant = AntData(ant_id, bdata)
    
    last_frame = max(clean_crops.index)
    default_closest_change = {'before':0, 'after':last_frame}
    
    if direction == 'before':
        end_frame = {'red':trop.start_frame-1,'yellow':trop.start_frame-1}
        all_measurements_before = clean_crops.loc[0:end_frame['red'],ant_id]
        start_frame = find_closest_change(all_measurements_before, direction, default_closest_change[direction])
        
    elif direction == 'after':
        start_frame = {'red':trop.end_frame+1, 'yellow':trop.end_frame+1}
        all_measurements_after = clean_crops.loc[start_frame['red']:,ant_id]
        end_frame = find_closest_change(all_measurements_after, direction, default_closest_change[direction])
    
    raw_measurements = {}
    for color in ['red','yellow']:
        raw_measurements[color] = ant.crop_dict_raw[color].loc[start_frame[color]:end_frame[color]]
    
    return raw_measurements
        

        
def find_closest_change(measurements, direction, default_closest_change):
    take_min_or_max = {'before':max, 'after':min}
    closest_change = {}
    for color in ['red','yellow']:
        diff_temp = measurements[color].diff()
        diff_temp.iloc[0]=0
        changes_temp = diff_temp != 0
        if changes_temp.any():
            closest_change[color] = take_min_or_max[direction](measurements[color][changes_temp].index)
        else:
            closest_change[color] = default_closest_change
    return closest_change


In [20]:
measurements_before = get_ant_measurements_before_or_after(trop.ants[1],trop,bdata,clean_crops,'before')
measurements_before

{'red': 1     NaN
 3     0.0
 5     0.0
 7     0.0
 9     0.0
 11    0.0
 13    NaN
 15    0.0
 17    0.0
 19    0.0
 21    0.0
 23    0.0
 25    0.0
 27    0.0
 29    0.0
 31    0.0
 33    0.0
 35    0.0
 37    0.0
 39    0.0
 41    0.0
 43    0.0
 45    0.0
 47    0.0
 49    0.0
 51    0.0
 53    0.0
 55    0.0
 57    0.0
 59    0.0
 61    0.0
 63    NaN
 65    0.0
 67    0.0
 69    0.0
 71    0.0
 Name: a76-crop_intensity, dtype: float64, 'yellow': 58    294778.236084
 60    275893.968750
 62    226689.101562
 64              NaN
 66    236148.515625
 68    233415.937500
 70    212696.437500
 Name: a76-crop_intensity, dtype: float64}

In [21]:
measurements_after = get_ant_measurements_before_or_after(trop.ants[1],trop,bdata,clean_crops,'after')
measurements_after

{'red': 117    0.0
 119    0.0
 121    NaN
 Name: a76-crop_intensity, dtype: float64, 'yellow': 116    38480.195312
 118    41384.394531
 120    23954.623047
 122             NaN
 124             NaN
 126             NaN
 128             NaN
 130             NaN
 132             NaN
 134             NaN
 136             NaN
 138             NaN
 140             NaN
 142             NaN
 144             NaN
 146             NaN
 148             NaN
 150             NaN
 152             NaN
 154             NaN
 156             NaN
 158             NaN
 160             NaN
 162             NaN
 164    13339.900391
 166    19553.306641
 168             NaN
 170    29089.476562
 172             NaN
 174    35045.824219
 176    39815.304688
 178             NaN
 180    30041.677734
 182             NaN
 184     6027.013672
 186        0.000000
 188        0.000000
 190        0.000000
 192        0.000000
 194        0.000000
 196        0.000000
 198        0.000000
 200        0.000000
 2

In [ ]:
get_ant_measurements_before(trop.ants[1],trop,bdata,clean_crops)

In [ ]:
ant1 = trop.ants[0]
ant1

In [ ]:
ant2 = trop.ants[1]
ant2

In [ ]:
#ant1_obj = AntData(ant1, bdata)

In [ ]:
clean_crops[ant2]

In [ ]:
trop.start_frame

In [ ]:
clean_crops.loc[0:trop.start_frame-1,ant2]

In [ ]:
ant2_yellow_before_all = clean_crops.loc[0:trop.start_frame-1,ant2]['yellow']

In [ ]:
max(ant2_yellow_before_all[ant2_yellow_before_all.diff()!=0].index)

In [ ]:
ant2_obj = AntData(ant2,bdata)

In [ ]:
%matplotlib qt
plt.plot(ant2_obj.crop_dict_raw['yellow'])
plt.plot(ant2_yellow_before_all)

In [ ]:
ant2_yellow_after_all = clean_crops.loc[trop.end_frame+1:,ant2]['yellow']
ant2_yellow_after_all

In [ ]:
min(ant2_yellow_after_all[ant2_yellow_after_all.diff()!=0].index)

In [ ]:
after_diff = ant2_yellow_after_all.diff()
after_diff.iloc[0]=0
min(ant2_yellow_after_all[after_diff!=0].index)

In [ ]:
plt.plot(ant2_yellow_after_all)

In [ ]:
ant2_obj.crop_dict_raw['yellow']